[NCBI Gene dataset](https://www.ncbi.nlm.nih.gov/datasets/gene/GCF_000001405.40/) of GRCh38.p14

## Build the genes table

In [ ]:
# start an empty df with columns
import pandas as pd
df = pd.DataFrame(columns=[
    "ncbi_gene_id", "symbol", "name", "type", "locus", "transcripts"
])
headers = []
with open("hg38_dataset.tsv") as f:
    for i, line in enumerate(f):
        # Name headers
        if i == 0: 
            headers = line.strip().split("\t")
            continue

        fields = line.strip().split("\t")

        # Make sure the gene id isn't already in the df -- if it is, we'll combine the transcripts
        gene_id = fields[headers.index("Gene ID")]
        if gene_id in df["ncbi_gene_id"].values:
            # I don't know how two rows could have the same Gene ID but no transcripts ?
            # They're V-segments? Biology is complicated
            try: 
                idx = df[df["ncbi_gene_id"] == gene_id].index[0]
                df.loc[idx, "transcripts"].append(fields[headers.index("Transcripts accession")])
            except:
                print("Error with gene id", gene_id)
                pass

        # Otherwise, add a new row
        else:
            row = {
                "ncbi_gene_id": fields[headers.index("Gene ID")],
                "symbol": fields[headers.index("Symbol")],
                "name": fields[headers.index("Name")],
                "type": fields[headers.index("Gene Type")],
                "locus": (fields[headers.index("Chromosome")], 
                        fields[headers.index("Orientation")],
                        int(fields[headers.index("Begin")]),
                        int(fields[headers.index("End")])),
            }
            # Some rows have no transcripts (pseudogenes)
            try:
                row["transcripts"] = [fields[headers.index("Transcripts accession")]]
            except:
                row["transcripts"] = []
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

# Save as tsv (safer for lists etc)
df.to_csv("hg38_genes.tsv", sep="\t", index=False)

## Build transcript tables

In [13]:
import pandas as pd

LINES_PER_FILE = 50000
input_fname = "./hg38_from_ncbi/genomic.gtf"
output_dir = "./server/data/hg38/test_transcripts/" # We'll add the file number and .tsv later
new_file_flag = False
big_file_index = {}
current_file_index = []

df_template = pd.DataFrame(columns=[
    "gene", "transcript", "transcript_bounds", 
    "CDSs", "exons", "start_codon", "stop_codon"])
df = df_template.copy()
with open(input_fname) as f:
    for i, line in enumerate(f):
        line = line.strip()
        if line.startswith("#"):
            continue

        # Build an attribute dictionary
        fields = line.strip().split("\t")
        attributes = fields[8].strip().split(";")
        attributes = attributes[:-1] # Remove the last empty string (ends with a semicolon)
        attributes_dict = {}
        for attr in attributes:
            key, value =  attr.strip().split(" ", 1)
            # There can be multiple values for a key...
            if key in attributes_dict:
                attributes_dict[key] = attributes_dict[key] + "," + value.replace('"', "")
            else:
                attributes_dict[key] = value.replace('"', "")

        # Build a fields dictionary
        headers = ["seqname", "source", "feature", "start", "end", "score", "strand", "frame"]
        fields_dict = dict(zip(headers, fields[:8]))
        
        # Now we do can build the row for the df
        transcript_id = attributes_dict["transcript_id"] # Will be empty string for genes
        if (transcript_id != "") and (transcript_id in df["transcript"].values):
        # New feature on an existing transcript
            current_index = df[df["transcript"] == transcript_id].index[0]
            current_row = df.loc[current_index].to_dict()
        
        elif (transcript_id != ""):
        # New transcript but same gene (presumably these are in order but who knows...)
        # Actually overlapping genes would cause a problem IF they're right at the boundary of files...
        # We'll just make sure that the same gene doesn't apear in multiple files :)
            current_row = {
                "gene": attributes_dict["gene_id"],
                "source": fields_dict["source"],
                "transcript": transcript_id,
                "product": attributes_dict["product"] if "product" in attributes_dict else None,
                "transcript_biotype": attributes_dict["transcript_biotype"],
                "xref": attributes_dict["db_xref"],
                "transcript_bounds": (fields_dict["start"], fields_dict["end"]),
                "CDSs": [],
                "exons": [],
                "start_codon": None,
                "stop_codon": None,
            }
            # Add the row to the df and continue
            df = pd.concat([df, pd.DataFrame([current_row])], ignore_index=True)
            current_index = df.index[-1]
            
        elif (transcript_id == "") and (fields_dict["feature"] == "gene") and (new_file_flag):
        # New gene and we've crossed the file boundary
            # First, move the current_file_index to the big_file_index
            current_file_number = len(big_file_index)
            big_file_index[current_file_number] = current_file_index
            # Then, save the current df
            df.to_csv(f"{output_dir}{current_file_number}.tsv", sep="\t", index=False)
            # Reset the df, and flag!, and current_file_index (don't forget to add the new gene to the index)
            df = df_template.copy()
            current_file_index = [attributes_dict["gene_id"]]
            new_file_flag = False
            continue

        elif (transcript_id == "") and (fields_dict["feature"] == "gene"):
        # New gene but will remain in the same file
            # Add the new gene to the index, but not the df (we only want transcripts)
            current_file_index.append(attributes_dict["gene_id"])
            continue
        else:
            raise ValueError(f"Unknown line: {line}")

        # Populate/update the current row depending on the feature
        feature = fields_dict["feature"]
        if (feature == "transcript"):
            current_row["transcript_bounds"] = (fields_dict["start"], fields_dict["end"])
        elif (feature == "CDS"):
            current_row["CDSs"].append((fields_dict["start"], fields_dict["end"]))
        elif (feature == "exon"):
            current_row["exons"].append((fields_dict["start"], fields_dict["end"]))
        elif (feature == "start_codon"):
            current_row["start_codon"] = (fields_dict["start"], fields_dict["end"])
        elif (feature == "stop_codon"):
            current_row["stop_codon"] = (fields_dict["start"], fields_dict["end"])
        else:
            raise ValueError(f"Unknown feature: {feature}")
            # feature should never be gene, I think?
        
        # Replace the row in the df
        df.loc[current_index] = current_row

        # Save every LINES_PER_FILE rows
        # But pair all transcripts with their gene, so just add a flag until we hit a new gene
        if i % LINES_PER_FILE == 0:
            new_file_flag = True
    
# Save the last file
current_file_number = len(big_file_index)
big_file_index[current_file_number] = current_file_index
df.to_csv(f"{output_dir}{current_file_number}.tsv", sep="\t", index=False)
big_file_index[current_file_number] = current_file_index # And append this to the big index

# Save the big index
reversed_file_index = {} # We want the gene to be the key
for key, value in big_file_index.items():
    for gene in value:
        reversed_file_index[gene] = key
with open(f"{output_dir}index.tsv", "w") as f:
    for key, value in reversed_file_index.items():
        f.write(f"{key}\t{value}\n")

AssertionError: 

In [18]:
# Whoops I forgot to add a header. Read the file and add it
index_fname = "./server/data/hg38/transcripts/index.tsv"
df = pd.read_csv(index_fname, sep="\t", header=None)
df.columns = ["gene_symbol", "file_index"]
df.to_csv(index_fname, sep="\t", index=False)


## Build genome sequence files

In [104]:
INPUT_FASTA = "./hg38_from_ncbi/hg38_genomic.fna"
OUTPUT_DIR = "./server/data/hg38/sequences/"

with open(INPUT_FASTA, "r") as f:
    current_chromosome = None
    current_sequence = []

    for line in f:
        line = line.strip()  # Remove any leading/trailing whitespace
        if line.startswith(">"):  # New header!
            if current_chromosome is not None:
                # Save the previous chromosome sequence to a file (will run every chromosome that's not the first)
                output_file = f"{OUTPUT_DIR}{current_chromosome}.txt"
                with open(output_file, "w", encoding="ascii") as out_f: # ASCII to make sure it's fixed width
                    out_f.write("".join(current_sequence))  # Write the sequence
                print(f"Written: {output_file}")

            current_chromosome = line[1:]  # Extract chromosome name (remove the >)
            current_sequence = []  # Reset the sequence list for the new chromosome
        else:
            current_sequence.append(line)  # Append sequence lines

    # Write the last chromosome if any
    if current_chromosome is not None:
        output_file = f"{OUTPUT_DIR}{current_chromosome}.txt"
        with open(output_file, "w", encoding="ascii") as out_f:
            out_f.write("".join(current_sequence))
        print(f"Written: {output_file}")


Written: ./server/data/hg38/sequences/NC_000001.11 Homo sapiens chromosome 1, GRCh38.p14 Primary Assembly.txt
Written: ./server/data/hg38/sequences/NT_187361.1 Homo sapiens chromosome 1 unlocalized genomic scaffold, GRCh38.p14 Primary Assembly HSCHR1_CTG1_UNLOCALIZED.txt
Written: ./server/data/hg38/sequences/NT_187362.1 Homo sapiens chromosome 1 unlocalized genomic scaffold, GRCh38.p14 Primary Assembly HSCHR1_CTG2_UNLOCALIZED.txt
Written: ./server/data/hg38/sequences/NT_187363.1 Homo sapiens chromosome 1 unlocalized genomic scaffold, GRCh38.p14 Primary Assembly HSCHR1_CTG3_UNLOCALIZED.txt
Written: ./server/data/hg38/sequences/NT_187364.1 Homo sapiens chromosome 1 unlocalized genomic scaffold, GRCh38.p14 Primary Assembly HSCHR1_CTG4_UNLOCALIZED.txt
Written: ./server/data/hg38/sequences/NT_187365.1 Homo sapiens chromosome 1 unlocalized genomic scaffold, GRCh38.p14 Primary Assembly HSCHR1_CTG5_UNLOCALIZED.txt
Written: ./server/data/hg38/sequences/NT_187366.1 Homo sapiens chromosome 1 unlo

In [6]:
input_fname = "./hg38_from_ncbi/genomic.gtf"
# read like 100 lines for now
lines = []
with open(input_fname) as f:
    for i, line in enumerate(f):
        if i > 100:
            break
        line = line.strip().replace("\t", "     ")
        lines.append(line)

lines

['#gtf-version 2.2',
 '#!genome-build GRCh38.p14',
 '#!genome-build-accession NCBI_Assembly:GCF_000001405.40',
 '#!annotation-date 08/23/2024',
 '#!annotation-source NCBI RefSeq GCF_000001405.40-RS_2024_08',
 'NC_000001.11     BestRefSeq     gene     11874     14409     .     +     .     gene_id "DDX11L1"; transcript_id ""; db_xref "GeneID:100287102"; db_xref "HGNC:HGNC:37102"; description "DEAD/H-box helicase 11 like 1 (pseudogene)"; gbkey "Gene"; gene "DDX11L1"; gene_biotype "transcribed_pseudogene"; pseudo "true";',
 'NC_000001.11     BestRefSeq     transcript     11874     14409     .     +     .     gene_id "DDX11L1"; transcript_id "NR_046018.2"; db_xref "GeneID:100287102"; db_xref "GenBank:NR_046018.2"; db_xref "HGNC:HGNC:37102"; gbkey "misc_RNA"; gene "DDX11L1"; product "DEAD/H-box helicase 11 like 1 (pseudogene)"; pseudo "true"; transcript_biotype "transcript";',
 'NC_000001.11     BestRefSeq     exon     11874     12227     .     +     .     gene_id "DDX11L1"; transcript_id "N

In [8]:
test = "abcdedfghijklmnopqrstuvwxyz"
test[0:3]

'abc'

In [1]:
# I want to know all the differnt biotypes in the gtf file
input_fname = "./hg38_from_ncbi/genomic.gtf"
biotypes = set()
with open(input_fname, "r") as f:
    for line in f:
        if line.startswith("#"):
            continue
        fields = line.strip().split("\t")
        attributes = fields[8].strip().split(";")
        attributes = attributes[:-1] # Remove the last empty string (ends with a semicolon)
        attributes_dict = {}
        for attr in attributes:
            key, value =  attr.strip().split(" ", 1)
            if key == "transcript_biotype":
                biotypes.add(value.replace('"', ""))

biotypes

{'C_gene_segment',
 'D_gene_segment',
 'J_gene_segment',
 'RNase_MRP_RNA',
 'RNase_P_RNA',
 'V_gene_segment',
 'Y_RNA',
 'antisense_RNA',
 'lnc_RNA',
 'mRNA',
 'miRNA',
 'ncRNA',
 'primary_transcript',
 'rRNA',
 'scRNA',
 'snRNA',
 'snoRNA',
 'tRNA',
 'telomerase_RNA',
 'transcript',
 'vault_RNA'}